In [ ]:
import torch
import matplotlib.pyplot as plt
import random

# Advanced Computer Science and Python Concepts

## Callbacks

Used a lot in event driven GUI

### Callbacks as GUI events

In [ ]:
import ipywidgets as widgets

From the [ipywidget docs](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Events.html):

- *the button widget is used to handle mouse clicks. The on_click method of the Button can be used to register function to be called when the button is clicked*

In [ ]:
w = widgets.Button(description='Click me')

In [ ]:
w

Button(description='Click me', style=ButtonStyle())

In [ ]:
def f(o): print('hi')

In [ ]:
w.on_click(f)

*NB: When callbacks are used in this way they are often called "events".*

### Creating your own callback

In [ ]:
from time import sleep

In [ ]:
def slow_calculation():
    """Slow like a model training :-)"""
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [ ]:
slow_calculation()

30

Would be nice to get feedback during the slow calculation

In [ ]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb: cb(i)
    return res

In [ ]:
def show_progress(epoch): print(f"Awesome! We've finished epoch {epoch}!")

In [ ]:
slow_calculation(show_progress)

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

### Lambdas and partials

Can create the callback in the slow_calculation function with lambda

In [ ]:
slow_calculation(lambda o: print(f"Awesome! We've finished epoch {o}!"))

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

Can design a more complicated callback that is customizable

In [ ]:
def show_progress(exclamation, epoch): print(f"{exclamation}! We've finished epoch {epoch}!")

In [ ]:
slow_calculation(lambda o: show_progress("OK I guess", o))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [ ]:
def make_show_progress(exclamation):
    def _inner(epoch): print(f"{exclamation}! We've finished epoch {epoch}!")
    return _inner

In [ ]:
slow_calculation(make_show_progress("Nice!"))
slow_calculation(make_show_progress("Great!"))

Nice!! We've finished epoch 0!
Nice!! We've finished epoch 1!
Nice!! We've finished epoch 2!
Nice!! We've finished epoch 3!
Nice!! We've finished epoch 4!
Great!! We've finished epoch 0!
Great!! We've finished epoch 1!
Great!! We've finished epoch 2!
Great!! We've finished epoch 3!
Great!! We've finished epoch 4!


30

In [ ]:
from functools import partial

In [ ]:
slow_calculation(partial(show_progress, "OK I guess"))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [ ]:
f2 = partial(show_progress, "OK I guess")

### Callbacks as callable classes

Callbacks do not have to be a function, they just need to be callable, so we can have callable class instead. It is cleaner, as we can set parameters in `__init__` and then it gets called

In [ ]:
class ProgressShowingCallback():
    def __init__(self, exclamation="Awesome"): self.exclamation = exclamation
    def __call__(self, epoch): print(f"{self.exclamation}! We've finished epoch {epoch}!")

In [ ]:
cb = ProgressShowingCallback("Just super")

In [ ]:
slow_calculation(cb)

Just super! We've finished epoch 0!
Just super! We've finished epoch 1!
Just super! We've finished epoch 2!
Just super! We've finished epoch 3!
Just super! We've finished epoch 4!


30

### Multiple callback funcs; `*args` and `**kwargs`

In [ ]:
def f(*args, **kwargs): 
    print(f"args: {args}; kwargs: {kwargs}")

In [ ]:
f(3, 'a', thing1="hello")

args: (3, 'a'); kwargs: {'thing1': 'hello'}


In [ ]:
def g(a,b,c=0): print(a,b,c)

In [ ]:
args = [1,2]
kwargs = {'c':3}
g(*args, **kwargs)

1 2 3


Smarter way to use and create callbacks 
- the callback class has several methods, which can be called at different speficic moments.
- the function using the optional callbacks will call each method at the appropriate time in the flow


In [ ]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):

        # run cb before the calculation, if any 
        if cb: cb.before_calc(i, arg=i*2)
        
        # perform the calculation
        res += i*i
        sleep(1)
        
        # run cb after the calculation, if any
        if cb: cb.after_calc(i, val=res)
    
    return res


In [ ]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs): print(f"About to start")
    def after_calc (self, *args, **kwargs): print(f"Done step")

In [ ]:
slow_calculation(cb=PrintStepCallback())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

The callback class also can include parameters to take from the main function, like `epoch`. 
> It is possible because the function slow_calculation is passing arguments to the callback function:
> `cb.before_calc(i, arg=i*2)` and `cb.after_calc(i, val=res)`

In [ ]:
class PrintStatusCallback():
    def __init__(self): pass
    def before_calc(self, epoch, arg, **kwargs):      print(f"About to start epoch: {epoch} - {arg}")
        
    def after_calc (self, epoch, val, **kwargs): print(f"After epoch {epoch}:        {val}")        

In [ ]:
slow_calculation(PrintStatusCallback())

About to start epoch: 0 - 0
After epoch 0:        0
About to start epoch: 1 - 2
After epoch 1:        1
About to start epoch: 2 - 4
After epoch 2:        5
About to start epoch: 3 - 6
After epoch 3:        14
About to start epoch: 4 - 8
After epoch 4:        30


30

### Modifying behavior

In [ ]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb,'before_calc'): 
            cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb and hasattr(cb,'after_calc'):
            if cb.after_calc(i, res):    
                print("stopping early")
                break
    return res

In [ ]:
class PrintAfterCallback():
    def after_calc (self, epoch, val):
        print(f"After {epoch}: {val}")
        if val>10: return True

In [ ]:
slow_calculation(PrintAfterCallback())

After 0: 0
After 1: 1
After 2: 5
After 3: 14
stopping early


14

In [ ]:
class SlowCalculator():
    def __init__(self, cb=None): self.cb,self.res = cb,0
    
    def callback(self, cb_name, *args):
        """Pick the correct callback function based on cb_name"""
        if not self.cb: return
        cb = getattr(self.cb,cb_name, None)
        if cb: return cb(self, *args)

    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            
            self.res += i*i
            sleep(1)
            if self.callback('after_calc', i):
                print("stopping early")
                break

In [ ]:
class ModifyingCallback():
    def after_calc (self, calc, epoch):
        print(f"After {epoch}: {calc.res}")
        if calc.res>10: return True
        if calc.res<3: calc.res = calc.res*2

In [ ]:
calculator = SlowCalculator(ModifyingCallback())

In [ ]:
calculator.calc()
calculator.res

After 0: 0
After 1: 1
After 2: 6
After 3: 15
stopping early


15

## `__dunder__` thingies

Anything that looks like `__this__` is, in some way, *special*. Python, or some library, can define some functions that they will call at certain documented times. For instance, when your class is setting up a new object, python will call `__init__`. These are defined as part of the python [data model](https://docs.python.org/3/reference/datamodel.html#object.__init__).

For instance, if python sees `+`, then it will call the special method `__add__`. If you try to display an object in Jupyter (or lots of other places in Python) it will call `__repr__`.

In [ ]:
class SloppyAdder():
    def __init__(self,o): self.o=o
    def __add__(self,b): return SloppyAdder(self.o + b.o + 0.01)
    def __repr__(self): return str(self.o)

In [ ]:
a = SloppyAdder(1)
b = SloppyAdder(2)
a+b

3.01

Special methods you should probably know about (see data model link above) are:

- `__getitem__`
- `__getattr__`
- `__setattr__`
- `__del__`
- `__init__`
- `__new__`
- `__enter__`
- `__exit__`
- `__len__`
- `__repr__`
- `__str__`

### `__getattr__` and `getattr`

In [ ]:
class A: 
    aa,bb=1,2

In [ ]:
a = A()

In [ ]:
a.aa, a.bb

(1, 2)

In [ ]:
getattr(a, 'aa'), getattr(a, 'bb')

(1, 2)

In [ ]:
getattr(a, 'bb' if random.random()>0.5 else 'aa')

1

We can define ourself how `getattr` should behave, by explicitely creating a **`__getattr__`** in our class. 

`__getattr__` is called for those attribute called that are not explicitely defined yet

In [ ]:
class B:
    aa, bb = 1, 2
    def __getattr__(self, k):
        if k[0] == '_': 
            raise AttributeError(f"{k} is not an attribute you can use")
        return f'Hello from {k}'

In [ ]:
b = B()

In [ ]:
b.aa, b.bb

(1, 2)

In [ ]:
b.foo, b.michael

('Hello from foo', 'Hello from michael')

In [ ]:
b.foo

'Hello from foo'

```python
b._blabla

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Input In [91], in <cell line: 1>()
----> 1 b._blabla

Input In [74], in B.__getattr__(self, k)
      3 def __getattr__(self, k):
      4     if k[0] == '_': 
----> 5         raise AttributeError(f"{k} is not an attribute you can use")
      6     return f'Hello from {k}'

AttributeError: _blabla is not an attribute you can use
```

In [ ]:
from fastcore.test import test_fail
def fn(): return b._blabla
test_fail(fn, contains='_blabla is not an attribute you can use')